# Recomendación de planes de telefonía

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. En este proyecto se desarrollará un modelo para recomendar uno de los dos nuevos planes de Megaline, Smart o Ultra, basándose en el comportamiento de los clientes.

Se entrenarán distintos modelos de clasificación a partir de un conjunto de datos de clientes que ya se han cambiado a los planes nuevos. Se evaluará la calidad de los modelos generados y se seleccionará el que presente una mayor exactitud.

Para los modelos de clasificación se utilizarán las funciones `DecisionTreeClassifier`, `RandomForestClassifier` y `LogisticRegression` de la librería `sklearn`.


**Tabla de contenidos**<a id='toc0_'></a>    
- [Información general](#toc1_)    
- [Entrenamiento y evaluación de modelos](#toc2_)    
  - [Segmentación de los datos](#toc2_1_)    
  - [Árbol de decisión](#toc2_2_)    
  - [Bosque Aleatorio](#toc2_3_)    
  - [Regresión logística](#toc2_4_)    
  - [Conclusiones](#toc2_5_)    
- [Comprobación de la calidad del mejor modelo con el conjunto de prueba](#toc3_)    
- [Prueba de cordura y conclusiones](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[Información general](#toc0_)

In [1]:
# Librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Lectura del dataset
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Los datos comprenden un conjunto de clientes que ya se cambiaron a uno de los nuevos planes, Smart o Ultra. De cada cliente se conoce el número de llamados, minutos empleados por llamada, mensajes de texto y datos de internet. Esta información será utilizada como los features de los modelos para predecir el target, que en este caso es la columna `is_ultra`.

## <a id='toc2_'></a>[Entrenamiento y evaluación de modelos](#toc0_)

### <a id='toc2_1_'></a>[Segmentación de los datos](#toc0_)

Al contar solo con el conjunto de datos fuente se dividirá este en tres partes según la proporción 3:1:1, para los conjuntos de entrenamiento, validación y prueba, respectivamente. 

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
# Primera división del dataset para obtener el conjunto de entrenamiento
features_train, features_rest, target_train, target_rest = train_test_split(
    features, target, test_size=0.4, random_state=12345)
# Segunda división del dataset para obtener los conjuntos de validación y prueba
features_valid, features_test, target_valid, target_test = train_test_split(
    features_rest, target_rest, test_size=0.5, random_state=12345)

In [5]:
# Se comprueba el tamaño de los conjuntos resultantes
print("Tamaño de los conjuntos y su proporción con respecto al conjunto original:")
print("")
print(f"Conjunto de entrenamiento: {len(features_train)}, {len(features_train)/len(df):.1%}")
print(f"Conjunto de validación: {len(features_valid)}, {len(features_valid)/len(df):.1%}")
print(f"Conjunto de prueba: {len(features_test)}, {len(features_test)/len(df):.1%}")
print(f"Conjunto original: {len(df)}, {len(df)/len(df):.1%}")

Tamaño de los conjuntos y su proporción con respecto al conjunto original:

Conjunto de entrenamiento: 1928, 60.0%
Conjunto de validación: 643, 20.0%
Conjunto de prueba: 643, 20.0%
Conjunto original: 3214, 100.0%


Habiendo segmentado los datos, se procede a entrenar y evaluar los modelos utilizando los conjuntos de entrenamiento y validación.

### <a id='toc2_2_'></a>[Árbol de decisión](#toc0_)

Se llama a la función `DecisionTreeClassifier()`. En los hiperparámetros, se fija `random_state=123456` y en un bucle se prueban distintos valores de `max_depth` (del 1 al 10) para conocer el que genere el modelo de mayor exactitud. 

In [6]:
best_model_dt = None
best_score = 0
best_depth = 0
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=123456, max_depth=depth)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_model_dt = model
        best_score = score
        best_depth = depth
print(f"Exactitud del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_score}")

Exactitud del mejor modelo en el conjunto de validación (max_depth = 3): 0.7853810264385692


Para el rango de valores de `max_depth` evaluado, la profundidad máxima de 3 produce el modelo de mayor exactitud, que es igual al 78.5%.

### <a id='toc2_3_'></a>[Bosque Aleatorio](#toc0_)

Se llama a la función `RandomForestClassifier()`. En los hiperparámetros, se fija `random_state=123456` y en un bucle anidado se prueban distintos valores de árboles o `n_estimatores` (del 10 a 50 en pasos de 10) y de `max_depth` (del 1 al 10) para conocer el que genere el modelo de mayor exactitud. 

In [7]:
best_model_rf = None
best_score = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10): #Ver que rango usar
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        score = model.score(features_valid, target_valid) 
        if score > best_score:
            best_model_rf = model
            best_score = score
            best_est = est
            best_depth = depth

print("Exactitud del mejor modelo en el conjunto de validación:", best_score, "n_estimators:", best_est, "best_depth:", best_depth)


Exactitud del mejor modelo en el conjunto de validación: 0.8087091757387247 n_estimators: 40 best_depth: 8


El modelo con mejor exactitud (80.9%) fue aquel configurado con un número de árboles de 40 y una profundidad máxima de 8.

### <a id='toc2_4_'></a>[Regresión logística](#toc0_)

Se llama a la función LogisticRegression(). Se fija `random_state=123456` y `solver='liblinear'`.

In [8]:
model = LogisticRegression(random_state=123456, solver='liblinear') 
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)
# print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)


Exactitud del modelo de regresión logística en el conjunto de validación: 0.7589424572317263


El modelo de regresión logística arrojó una exactitud del 75.9%, la más baja de todos los modelos.

### <a id='toc2_5_'></a>[Conclusiones](#toc0_)

Si bien los tres tipos de modelo lograron resultados razonables, el modelo de bosque aleatorio o Random Forest Classifier obtuvo la mayor exactitud, alcanzando un 80,9%. Esto indica que este modelo puede ser el más adecuado para abordar el caso específico de recomendar planes de telefonía.

## <a id='toc3_'></a>[Comprobación de la calidad del mejor modelo con el conjunto de prueba](#toc0_)

Al haber determinado el modelo con la mayor exactitud, se procede a evaluarlo con el conjunto de prueba que se segmentó anteriormente.

In [9]:
print(f'Exactitud: {best_model_rf.score(features_test, target_test)}')

Exactitud: 0.7962674961119751


La exactitud resultante de 79.6% es cercana a la exactitud obtenida con el conjunto de validación. Por lo tanto, el modelo tiene buenos resultados con otros conjuntos de datos.

## <a id='toc4_'></a>[Prueba de cordura y conclusiones](#toc0_)

Se realiza una prueba de cordura para asegurarse de que el modelo funciona mejor que la casualidad. Para ello se toma de nuevo el conjunto de prueba y se determina la proporción de clientes de cada plan frente al total del conjunto.

In [10]:
print(f'Usuarios de Smart: {sum(target_test == 0)}, {sum(target_test == 0)/len(target_test):.1%}')
print(f'Usuarios de Ultra: {sum(target_test == 1)}, {sum(target_test == 1)/len(target_test):.1%}')
print(f'Total de usuarios: {len(target_test)}, {len(target_test)/len(target_test):.1%}')

Usuarios de Smart: 440, 68.4%
Usuarios de Ultra: 203, 31.6%
Total de usuarios: 643, 100.0%


Los usuarios de Smart son mayoría en el conjunto con un 68.4%. Si se tuviera un modelo que solo predijera el tipo de plan más frecuente, en este caso Smart, este acertaría el 68.4% de las veces. El modelo de bosque aleatorio que se desarrolló antes acertó el 79,6% de las veces con el mismo conjunto. Por lo tanto, este modelo pasa la prueba de cordura al funcionar mejor que la casualidad.